### Natural Language Processing
      Assignment : 2

In [1]:
#Manha Zamir - 17L-4096

In [21]:
import csv
import spacy
from collections import Counter
import math
import random

#### Reading Training File

In [2]:
with open('Train-perplexity.txt', 'rt', encoding="utf-8") as f:
    data = f.read().rstrip()    
f.close()

#### Tokenizing the corpus to a list of words

In [3]:
data = data.lower()
nlp = spacy.load("en_core_web_sm")     #Using spacy to tokenize
doc = nlp(data)
tokens = []
for token in doc:
    tokens.append(token.text.strip())
tokens = list(filter(None, tokens))

punctuations = ['.', ',', ':', '?', '!', ';', '-', '\"']
for word in tokens:
    if word in punctuations:
        tokens.remove(word)
# print(tokens)
print(len(tokens))
set_of_tokens = set(tokens)   #Unique words
print(len(set_of_tokens))

1542
658


#### Generate n-gram models
    Unigram, Bigram, Trigram

In [4]:
unigrams, bigrams, trigrams = [], [], []
bigram_div, trigram_div  = [], []    

#Unigram
for index in tokens:

    unigrams.append(index)     #Generating unigrams
# print(unigrams)

#Bigram
for index in range(len(tokens)-1):

    b_first = tokens[index]
    bigram_div.append(b_first)  #Used later to calculate probabilities of bigram (eg: P(I am/ I), where 'I' is bigram_div)
    b_second = tokens[index+1]
    bi = b_first + " " + b_second
    bigrams.append(bi)        #Generating bigrams
# print(bigrams)

#Trigram
for index in range(len(tokens)-2):

    t_first = tokens[index]
    t_second = tokens[index+1]
    trigram_div.append(t_first + " " + t_second) #Used later to calculate trigram probabilities (eg: P(I am new/ I am), where 'I am' is trigram_div)
    t_third = tokens[index+2]
    tri = t_first + " " + t_second + " " + t_third
    trigrams.append(tri)     #Generating unigrams    
# print(trigrams)
    
print(len(unigrams))
print(len(bigrams))
print(len(trigrams))

1542
1541
1540


#### Frequency of each combination of models

In [5]:
unigram_freq = dict(Counter(unigrams))
bigram_freq = dict(Counter(bigrams))
trigram_freq = dict(Counter(trigrams))

print(len(unigram_freq))
print("Unigram Frequency\n")
print(unigram_freq)

print(len(bigram_freq))
print("\nBigram Frequency\n")
print(bigram_freq)

print(len(trigram_freq))
print("\nTrigram Frequency\n")
print(trigram_freq)


658
Unigram Frequency

{'to': 23, 'this': 10, 'urn': 1, 'let': 1, 'those': 1, 'repair': 1, 'that': 17, 'are': 5, 'either': 1, 'true': 3, 'or': 2, 'fair': 1, 'for': 6, 'these': 6, 'dead': 1, 'birds': 1, 'sigh': 1, 'a': 20, 'prayer': 1, 'sir': 1, 'charles': 1, 'into': 4, 'my': 12, 'chamber': 1, 'coming': 1, 'in': 38, 'when': 2, 'i': 19, 'was': 2, 'writing': 1, 'of': 67, 'fairy': 3, 'queen': 4, 'praysaid': 1, 'hewhen': 1, 'mab': 2, 'you': 8, 'do': 1, 'see': 1, 'present': 2, 'service': 2, 'her': 7, 'majesty': 1, 'and': 68, 'tell': 4, 'have': 3, 'heard': 2, 'fames': 1, 'loud': 1, 'report': 1, 'both': 2, 'beauty': 1, 'stately': 1, 'court': 2, 'within': 1, 'fancy': 2, 'viewed': 1, 'thoughts': 2, 'bowed': 1, 'low': 1, 'fearing': 1, 'should': 9, 'be': 2, 'rude': 2, 'kissing': 1, 'garment': 1, 'thin': 1, 'which': 4, 'made': 2, 'knelt': 1, 'upon': 6, 'thought': 1, 'like': 8, 'one': 1, 'prayed': 2, 'then': 4, 'whispers': 1, 'soft': 1, 'did': 2, 'his': 10, 'humble': 2, 'mirth': 1, 'sent': 1, 'thus'

#### Probabilities of n-grams

In [6]:
unigram_prob, bigram_prob, trigram_prob = {}, {}, {}
smoothed_unigram, smoothed_bigram, smoothed_trigram = {}, {}, {}  #Smoothed n-grams to use if unknowns found

#Unigram
for i in unigram_freq:
    unigram_prob[i] = unigram_freq[i]/len(tokens)
    smoothed_unigram[i] = (unigram_freq[i] + 0.8)/(len(tokens) + (0.8 * len(unigram_freq)))   #Smoothed unigram probabilities using Add-k Laplace 
print(unigram_prob)

#Bigram
for i, j in zip(bigrams, bigram_div):
    bigram_prob[i] = bigram_freq[i]/unigram_freq[j]
    smoothed_bigram[i] = (bigram_freq[i] + 0.8)/(unigram_freq[j] + (0.8 * len(bigram_freq)))   #Smoothed bigram probabilites using Add-k Laplace 
print(bigram_prob)  


#Trigram
for i, j in zip(trigrams, trigram_div):
    trigram_prob[i] = trigram_freq[i]/bigram_freq[j]
    smoothed_trigram[i] = (trigram_freq[i]+ 0.8)/(bigram_freq[j] + (0.8 * len(trigram_freq)))   #Smoothed trigram probabilites using Add-k Laplace 
# print(trigram_prob)

{'to': 0.014915693904020753, 'this': 0.00648508430609598, 'urn': 0.0006485084306095979, 'let': 0.0006485084306095979, 'those': 0.0006485084306095979, 'repair': 0.0006485084306095979, 'that': 0.011024643320363165, 'are': 0.00324254215304799, 'either': 0.0006485084306095979, 'true': 0.0019455252918287938, 'or': 0.0012970168612191958, 'fair': 0.0006485084306095979, 'for': 0.0038910505836575876, 'these': 0.0038910505836575876, 'dead': 0.0006485084306095979, 'birds': 0.0006485084306095979, 'sigh': 0.0006485084306095979, 'a': 0.01297016861219196, 'prayer': 0.0006485084306095979, 'sir': 0.0006485084306095979, 'charles': 0.0006485084306095979, 'into': 0.0025940337224383916, 'my': 0.007782101167315175, 'chamber': 0.0006485084306095979, 'coming': 0.0006485084306095979, 'in': 0.02464332036316472, 'when': 0.0012970168612191958, 'i': 0.01232166018158236, 'was': 0.0012970168612191958, 'writing': 0.0006485084306095979, 'of': 0.04345006485084306, 'fairy': 0.0019455252918287938, 'queen': 0.002594033722

#### Poetry generation for Unigram

In [7]:
gen_sentences, unigram_generation, uni_poem = [], [], ""

for sentences in range(0,3):
    persentence = []
    start = random.choice(unigrams)      #Starting each sentence with random unigram
    persentence.append(start)
    for words in range(random.randint(6,9)):
        word = random.choice(unigrams)   #Choosing random unigrams each time since unigrams do not utilize previous history of words
        persentence.append(word)
    gen_sentences.append(persentence)    #Generating list of new sentences

for i in gen_sentences:
    unigram_generation.append(' '.join(i))
    
for poem in unigram_generation:
    print(poem)                          #Generated poem using unigram
    uni_poem += poem + " "

and the these loves head of impe and throughout themes
thy their off a pipe and great hiawatha northern people
the stone walls bushes corn outward stooping excels land


#### Poetry generation for Bigram

In [8]:
bgram_sentences, bigram_generation, bipoem = [], [], []

for sentences in range(0,3):    
    
    persentence = []
    start = random.choice(unigrams)      #Starting each sentence with random unigram
    persentence.append(start)
    bp = bigram_prob.keys()
    for word in range(random.randint(6,9)): 
        maxprob = {}
        for i in bp:
            dp = i.split()          
            if start == dp[0]:         #If previous word is present in bigram probability dict, then store all keys having that word
                maxprob[i] = None
        for prob in maxprob:           
            if prob in bigram_prob.keys():   #Store values of all keys containing prev words
                maxprob[prob] = bigram_prob[prob]
        if maxprob:
            max_key = max(maxprob, key=maxprob.get)   #Select the key with maximum probability 
            start = max_key.split()[1]                
            persentence.append(start)                 #Append new word in the sentence

    bgram_sentences.append(persentence)               #Generating list of new sentences
               
for i in bgram_sentences:
    bigram_generation.append(' '.join(i)) 
    
for poem in bigram_generation:
    print(poem)                                      #Generated poem using bigram
    bipoem.append(poem)


it kindled in the forest with the forest with
stone quarry gitche manito the forest with the forest with
like a while hiawatha ye who love the


#### Poetry generation for Trigram

In [9]:
tgram_sentences, trigram_generation, tripoem = [], [], []

for sentences in range(0,3):    
    
    persentence = []
    start = random.choice(bigrams)      #Starting each sentence with random bigram
    persentence.append(start)
    tp = trigram_prob.keys()
    
    for word in range(random.randint(6,9)): 
        maxprob = {}
        for i in tp:
            dp = i.split()
            lookup = dp[0]+ " " + dp[1]
            
            if start == lookup:       #If previous word is present in trigram probability dict, then store all keys having that word
                maxprob[i] = None
        for prob in maxprob:
            if prob in trigram_prob.keys():   #Store values of all keys containing prev words
                maxprob[prob] = trigram_prob[prob]
        if maxprob:
            max_key = max(maxprob, key=maxprob.get)   #Select the key with maximum probability
            word_to_add = max_key.split()[2]
            start = ' '.join(max_key.split()[1:])
            persentence.append(word_to_add)           #Append two new words in the sentence

    tgram_sentences.append(persentence)              #Generating list of new sentences
               
for i in tgram_sentences:
    trigram_generation.append(' '.join(i))
    
for poem in trigram_generation:                      #Generated poem using bigram
    print(poem)    
    tripoem.append(poem)

    

kissing her garment thin which fancy made i
you bring triumphant mart in loves and gentle jollities arrayd after
" ye who love the haunts of nature love the haunts


#### Perplexity of newly generated poems using n-grams

In [10]:
def uniperplexity(data):
    sum = 0
    boolean = False
    s = uni_poem.split()      #Split into sentences
    for x in s:
        if x not in bigram_prob.keys():   #For each sentence, if word is not in unigram probability dict (unknown word), then set boolean = True, meaning we need to apply smoothing
            boolean = True
            
    for unitok in s:
        if boolean == True:         #If true, apply prob with smoothed values, else simple probablistic values      
            if unitok not in unigram_prob.keys():   #If word is not in unigram probability, apply smoothing to this unknown word (0 + 1/length of unigram dictionary)
                logs = 0.8/(0.8 * len(unigram_prob))
            else: logs = smoothed_unigram[unitok]   #If word present, use the smoothed probabilistic bigrams
        else: logs = unigram_prob[unitok]
        prob = math.log(logs)                       #Apply perplexity
        sum += prob
    pp = (-1/len(s)) * sum 
    return pp

In [11]:
def biperplexity(data):
    sum = 0
    boolean = False
    pp_bigrams = []
            
    for i in data:
        s = i.split()                       #Split into sentences
        for index in range(len(s)-1):
            pp_bigrams.append(s[index] + " " + s[index+1])   #Generated bigrams
            
    for x in pp_bigrams:
        if x not in bigram_prob.keys():    #For each sentence, if word is not in bigram probability dict (unknown word), then set boolean = True, meaning we need to apply smoothing
            boolean = True

    for bitok in pp_bigrams:
        if boolean == True:                #If true, apply prob with smoothed values, else simple probablistic values
            if bitok not in bigram_prob.keys():     #If word is not in bigram probability, apply smoothing to this unknown word (0 + 1/length of bigram dictionary)
                logs = 0.8/(0.8 * len(bigram_prob))
            else:
                logs = smoothed_bigram[bitok]       #If word present, use the smoothed probabilistic bigrams
        else: logs = bigram_prob[bitok]
            
        prob = math.log(logs)                      #Apply perplexity
        sum += prob

    pp = (-1/len(pp_bigrams)) * sum
    return pp

In [12]:
def triperplexity(data):
    sum = 0
    boolean = False
    pp_trigrams = []
    for i in data:
        s = i.split()                   #Split into sentences
        for index in range(len(s)-2):
            pp_trigrams.append(s[index] + " " + s[index+1] + " " + s[index+2])   #Generated trigrams
            
    for x in pp_trigrams:
        if x not in bigram_prob.keys():  #For each sentence, if word is not in trigram probability dict (unknown word), then set boolean = True, meaning we need to apply smoothing
            boolean = True               
    
    for tritok in pp_trigrams:
        if boolean == True:              #If true, apply prob with smoothed values, else simple probablistic values
            if tritok not in trigram_prob.keys():
                logs = 0.8/(0.8 * len(trigram_prob))        #If word is not in trigram probability, apply smoothing to this unknown word (0 + 1/length of trigram dictionary)
            else:
                logs = smoothed_trigram[tritok]   #If word present, use the smoothed probabilistic bigrams

        else: logs = trigram_prob[tritok]
        prob = math.log(logs)
        sum += prob                               #Apply perplexity        
        
    pp = (-1/len(pp_trigrams)) * sum
    return pp

In [13]:
#Unigram Perplexity
uni_perplexity = uniperplexity(uni_poem)
uni_perplexity

5.773227816638515

In [14]:
#Bigram Perplexity
bi_perplexity = biperplexity(bipoem)
bi_perplexity

1.4798190543648453

In [15]:
#Trigram Perplexity
tri_perplexity = triperplexity(tripoem)
tri_perplexity

6.460831683987577

#### Calculating Perplexity of Test data using Trained n-grams

#### Reading test file

In [16]:
with open('Test-perplexity.txt', 'rt', encoding="utf-8") as r:
    sentences = r.readlines()
r.close()

#### Tokenize data

In [17]:
newsen, s, tokens = [], [], []
nlp = spacy.load("en_core_web_sm")
for sentence in sentences:
    sentence = sentence.lower()
    doc = nlp(sentence)
    
    for token in doc:
        tokens.append(token.text)

punctuations = ['.', ',', ':', '?', '!', ';', '-', '\"']
for word in tokens:
    if word in punctuations:       
        tokens.remove(word)         #Cleaning data
        
m = ' '.join(tokens)
s.append(m.split('\n'))             #Splitting cleaned into sentences
for i in s[0]:
    newsen.append(i.strip())        
print(newsen)

['with their weapons and their war gear', 'painted like the leaves of autumn', 'painted like the sky of morning', 'wildly glaring at each other', 'in their faces stern defiance', 'in their hearts the feuds of ages', 'the hereditary hatred', 'the ancestral thirst of vengeance', 'gitche manito the mighty', 'the creator of the nations', 'looked upon them with compassion', 'with paternal love and pity', 'looked upon their wrath and wrangling', 'but as quarrels among children', 'but as feuds and fights of children', 'over them he stretched his right hand', 'to subdue their stubborn natures', 'to allay their thirst and fever', 'by the shadow of his right hand', 'spake to them with voice majestic', 'as the sound of far off waters', 'falling into deep abysses', 'warning chiding spake in this wise', 'o my children my poor children', 'listen to the words of wisdom', 'listen to the words of warning', 'from the lips of the great spirit', 'from the master of life who made you', 'i have given you la

#### Perplexities of ngrams

In [18]:
test_uni_pp = uniperplexity(newsen)
test_bi_pp = biperplexity(newsen)
test_tri_pp = triperplexity(newsen)

#Unigram Perplexity
test_uni_pp

5.773227816638515

In [19]:
#Bigram Perplexity
test_bi_pp

6.921422180125618

In [20]:
#Trigram Perplexity
test_tri_pp

7.241755906219141